In [25]:
from dowhy import CausalModel
import pandas as pd
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [20]:
# 1) Daten einlesen
path_validation = r"..\syntheticData\RealDataMerged\mergedManupulated.xlsx"
data_input = pd.read_excel(path_validation)

In [22]:
# 2) CausalModel aufsetzen
G = nx.DiGraph()
edges = [
    ("Rüstzeit", "Rüst- Bearbeitungszeit"),
    ("Bearbeitungszeit", "Rüst- Bearbeitungszeit"),
    ("Rüst- Bearbeitungszeit", "Soll/Ist Durchlaufszeit(t)"),
    ("Soll/Ist Durchlaufszeit(t)", "Abweichung (Tage)"),
    ("DurchlaufszeitSoll(t)", "Abweichung (Tage)"),
    ("Abweichung (Tage)", "Termintreue")
]

G.add_edges_from(edges)

model = CausalModel(
    data=data_input,
    treatment="DurchlaufszeitSoll(t)",
    outcome="Termintreue",
    graph=G
)


In [24]:
# 3) Identifizieren und schätzen
estimand = model.identify_effect()
print(estimand)

estimate = model.estimate_effect(
    estimand,
    method_name="backdoor.linear_regression"
)
print("Estimated effect:", estimate.value)

Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
           d                            
────────────────────────(E[Termintreue])
d[DurchlaufszeitSoll(t)]                
Estimand assumption 1, Unconfoundedness: If U→{DurchlaufszeitSoll(t)} and U→Termintreue then P(Termintreue|DurchlaufszeitSoll(t),,U) = P(Termintreue|DurchlaufszeitSoll(t),)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: frontdoor
Estimand expression:
 ⎡         d                                   d                               ↪
E⎢────────────────────(Termintreue)⋅────────────────────────([Abweichung (Tage ↪
 ⎣d[Abweichung (Tage)]              d[DurchlaufszeitSoll(t)]                   ↪

↪    ⎤
↪ )])⎥
↪    ⎦
Estimand assumption 1, Full-mediation: Abweichung (Tage) intercepts (blocks) all directed paths from DurchlaufszeitSoll(t) to T,e,r,m,i,n,t,r,e,u,e.
Estimand assumption 2, First-stage-unconfoundedness: I

In [27]:
# 5) Refutation (Placebo)
refute = model.refute_estimate(
    estimand, estimate,
    method_name="placebo_treatment_refuter",
    placebo_type="permute"
)
print(refute)

Refute: Use a Placebo Treatment
Estimated effect:-0.0017247012757206637
New effect:4.917304279738355e-05
p value:0.9199999999999999

